In [1]:
import os
import pandas as pd
import json, csv, sys
from pandas.io.json import json_normalize
import time
import datetime


# Import ProductBarcode

In [2]:
def getJsonFileNameList(mydir):

    fileNameList =os.listdir(mydir)
    jsonFileNameList=[]
    for file in fileNameList:
        if file[-4:] == 'json':
            jsonFileNameList.append(file)
        
    return jsonFileNameList

def json2dataframe(mydir, jsonFile):
    with open(mydir + jsonFile,encoding='UTF8') as f:
        jsonData = json.load(f) # list 얻는다.
        
    df = pd.DataFrame.from_dict(json_normalize(jsonData), orient='columns')
    return df

In [6]:
mydir = 'C:/Users/dongm/COSMOCHAIN/Data/Product/상공회의소/'
dir_output = 'C:/Users/dongm/COSMOCHAIN/Data/'

jsonFileNameList = getJsonFileNameList(mydir)
print('파일 목록')
print(jsonFileNameList)

파일 목록
['ProductBarcode.json']


In [7]:
dflist=[]
for jsonFile in jsonFileNameList:
    print('change <',jsonFile, '> to Dataframe')
    dflist.append(json2dataframe(mydir,jsonFile))
    
    

change < ProductBarcode.json > to Dataframe


In [11]:
barcode = dflist[0]
barcode = barcode[['BARCODE','PRNM','CLASSCODE','CLASSNAME4']]
barcode = barcode.sort_values(by=['BARCODE'])
barcode = barcode.reset_index(drop=True)
display(barcode.tail())


,BARCODE,PRNM,CLASSCODE,CLASSNAME4
477911,9556006011058,PH5.5 250ML,03200199,기타목욕용화장품
477912,9556006060247,JB Bedtime bath 1000ml,03200199,기타목욕용화장품
477913,9771210107001,"이스틴 틴트 루즈 1,2",03200499,기타아이메이크업용품
477914,9809169793317,다이알 선물세트,03200205,핸드워시
477915,9909208059210,쥬리아 알로에 베라 수딩젤,03209999,기타화장품


# Import 상품데이터 Master

In [ ]:
filename = '상품데이터 Master.xlsx'
df = pd.read_excel(mydir + filename, sheet_name = 3)
display(df.columns)

In [29]:
master = df[['브랜드명 (한글)','상품명 (한글)','원상품명']]

# 핵심부분 : 어떻게 유사도 계산할것인가?
### 1. 공백 제거
### 2. 모두 대문자로 변경
### 3. master의 상품명 모두 포함

In [30]:
# Master 파일의 상품명, trim
master_products = list(master['상품명 (한글)'])
master_products_trim = [product.replace(" ","").upper() for product in master_products]
total_len = len(master_products)

# BARCODE 파일의 바코드와 상품명, trim
barcode_barcode = list(barcode['BARCODE'])
barcode_prnms = list(barcode['PRNM'])
barcode_prnms_trim = [prnm.replace(" ","").upper() for prnm in barcode_prnms]



In [55]:
start_time = time.time()
now = datetime.datetime.now()
print('--- 시작 :',now, '---')
print('--- Master Data 개수 :',total_len,'---')

# 매칭 결과를 저장하는 list
PRNM = []
BARCODE = []


for idx, prod in enumerate(master_products_trim):
    for f, prnm in enumerate(barcode_prnms_trim):
        if prod in prnm:
            PRNM.append(barcode_prnms[f])
            BARCODE.append(barcode_barcode[f])
            break
    else:
        PRNM.append('#')
        BARCODE.append('#')
        
    if idx % (total_len//200) == 0:
        print("%3.1f 퍼센트 진행중" % (idx / total_len * 100))
    if idx == total_len//100:
        print("--- 예상 소요 시간 : %0.2f seconds" % float((time.time() - start_time)*100), '---')
    if idx > 201:
        break
    
now = datetime.datetime.now()
print("--- 전체 소요 시간 : %0.2f seconds ---" % (time.time() - start_time))
print('--- 종료 :',now, '---')


--- 시작 : 2019-01-09 18:56:06.729373 ---
--- Master Data 개수 : 18048 ---
0.0 퍼센트 진행중
0.5 퍼센트 진행중
1.0 퍼센트 진행중
--- 예상 소요 시간 : 1934.83 seconds ---
--- 전체 소요 시간 : 21.50 seconds ---
--- 종료 : 2019-01-09 18:56:28.232315 ---


# 매칭 개수 확인

In [ ]:
# test용으로 짧게 했을 때 len(PRNM)만큼만 사용
master = master.head(len(PRNM))
master['BARCODE']=BARCODE
master['PRNM']=PRNM

brands = master.groupby('브랜드명 (한글)')
brands = [brands.get_group(x) for x in brands.groups]

sum1=0
sum2=0
for brand in brands:
    matched = len(set(brand['PRNM'])) - 1
    brand_total = len(brand)
    print(brand['브랜드명 (한글)'].iloc[0]+' : '+str(matched)+'/'+str(brand_total))
    print('-*-'*10)
    sum1+=matched
    sum2+=brand_total

print(sum1)
print(sum2)

# Export to excel

In [34]:
master.to_excel(dir_output+'공백제거 후 일치.xlsx')